In [1]:
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt

In [2]:
image = cv2.imread('Resources/bill.jpg')
ratio = image.shape[0] / 500.0
image = imutils.resize(image, height = 500)
im_copy = image.copy()

In [3]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blurred, 190, 255, cv2.THRESH_BINARY)[1]
eroded = cv2.erode(thresh, None, iterations = 5)
dilated = cv2.dilate(eroded, None, iterations = 10)
edged = cv2.Canny(dilated, 50, 200)

In [4]:
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey()

-1

In [9]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    # determine the most extreme points along the contour
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    # if our approximated contour has four points, then we can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break

# show the contour (outline) of the piece of paper
cv2.drawContours(im_copy, [c], -1, (0, 255, 255), 2)

cv2.circle(im_copy, extLeft, 8, (0, 0, 255), -1)
cv2.circle(im_copy, extRight, 8, (0, 255, 0), -1)
cv2.circle(im_copy, extTop, 8, (255, 0, 0), -1)
cv2.circle(im_copy, extBot, 8, (255, 255, 0), -1)

cv2.imshow('Image copy', im_copy)
cv2.waitKey()

-1

In [10]:
# Perspective Transform & Threshold

In [11]:
def perspectiveTransform():
    p1 = np.array([extLeft, extBot, extTop, extRight], dtype=np.float32)
    p2 = np.array([[0,0],
                   [0,500],
                   [500,0],
                   [500,500]], dtype=np.float32)

    T = cv2.getPerspectiveTransform(p1, p2)
    imgTrans = cv2.warpPerspective(im_copy, T, (500, 500))

    cv2.imshow("Image", image)
    cv2.imshow("Transformed Image", imgTrans)
    cv2.waitKey()

perspectiveTransform()